# `backend.py`

# `basis.py`

# `coils.py`

# compute

## Introduction

In DESC, quantities of interest (such as force error `F`, magnetic field strength `|B|` and its vector components)   are computed from `Equilibrium` objects through the use of `compute_XXX` functions. The `Equilibrium` object has a `eq.compute(name="NAME OF QUANTITY TO COMPUTE",grid=Grid)` method which takes in the string `name` of the quantity to compute from the `Equilibrium`, and a `Grid` of coordinates $(\rho,\theta,\zeta)$ to evaluate the quantity at. This method then uses the `name` to find which `compute_XXX` function is needed to call in order to calculate that quantity.

These `compute_XXX` functions live inside of the `desc/compute` folder, and inside that folder the `data_index.py` file contains the list of all available quantities that `name` could specify to be computed, as well as information on that quantity and which `compute_XXX` function should be called in order to compute it.

The `compute_XXX` functions have function signatures that take in as arguments the necessary variables from `{R_lmn, Z_lmn, L_lmn, i_l, p_l,Psi}` required to calculate the quantity, as well as the `Transform` and/or `Profile` objects needed to evaluate those variables (which are spectral coefficients, except for `Psi`) at the points in real space specified by the desired `Grid` object (which is contained in the `Transform` objects passed).


An example compute function from `_field.py` is shown here for computing the magnetic field magnitude:
```python
def compute_magnetic_field_magnitude(
    R_lmn,
    Z_lmn,
    L_lmn,
    i_l,
    Psi,
    R_transform,
    Z_transform,
    L_transform,
    iota,
    data=None,
    **kwargs,
):
```
The first 3 arguments `R_lmn,Z_lmn,L_lmn` are the Fourier-Zernike spectral coefficients of the toroidal coordinates of the flux surfaces `R,Z`, and of the poloidal stream function $\lambda$. `i_l` is the coefficients of the rotational transform profile (typically a power series in `rho`). `Psi` is the enclosed toroidal flux.

The `_transform` arguments are `Transform` objects which transform the spectral coefficients to their values in real-space. `iota` is a `Profile` object which transforms `i_l` into its values in real-space.
The `data` argument is an optional dictionary. The compute functions store the quantities they calculate in a `data` dictionary and return it, and if `data` is passed in, the quantities this function computes will be added to this dictionary. This way, a compute function can add on to the quantities already calculated by previous compute functions, or it can use other compute functions to calculate preliminary quantities to avoid duplicating code. As an example, in the `compute_magnetic_field_magnitude` function, the contravariant components of the magnetic field $B^i$ are required, along with the metric coefficients $g_ij$. To calculate these, the function calls:

```python
data = compute_contravariant_magnetic_field(
    R_lmn,
    Z_lmn,
    L_lmn,
    i_l,
    Psi,
    R_transform,
    Z_transform,
    L_transform,
    iota,
    data=data,
)
data = compute_covariant_metric_coefficients(
    R_lmn, Z_lmn, R_transform, Z_transform, data=data
)
```

in order to populate `data` with these necessary preliminary quantities.


- talk about what the data arg is and how it is used
- maybe include example of how to make your own (let's say for a stupid thing like B_theta * B_zeta)





## Calculating Quantities

Inside the compute function, every quantity is stored inside the `data` dictionary under the key of the name of the quantity. 
As an example, `data['|B|']` contains the magnetic field magnitude.
This quantity is stored as a JAX array of size `(num_nodes,)` (if the quantity is NOT a vector), or `(num_nodes,3)` (if the quantity is a vector, i.e. `data['B']`, which contains all three components $[B_R, B_{\phi},B_{Z}]$ of $B$ at each node) (`num_nodes` is the number of nodes in the `Grid` object that the quantity was computed on. Can be accessed by `grid.num_nodes`). 
This array is flattened, so if the grid used has 10 equispaced gridpoints in $(\rho,\theta,\zeta)$, the grid will have $10^3 = 1000$ nodes, and any quantity calculated on that grid will be returned as an array of size `(num_nodes,)` if not a vector and `(num_nodes,3)` if a vector quantity.
### Scalar Algebra
Storing the quantities in arrays like this enables for easy computation using these quantities. For example, if one wants to calculate the magnitude of the pressure gradient $|\nabla p(\rho)| = \sqrt{p'(\rho)^2}|\nabla\rho|$, one simply [writes out the expression](https://github.com/PlasmaControl/DESC/blob/6d03cb015701b27d651bf804d36032c35119c536/desc/compute/_equil.py#L114) after calling the necessary compute functions:

```python
data["|grad(p)|"] = jnp.sqrt(data["p_r"] ** 2) * data["|grad(rho)|"]
```

### Vector Algebra
If calculating a quantity which involves vector algebra, the format of these arrays makes it simple to write out as well. As an example, if calculating the contravariant radial basis vector $\mathbf{e}^{\rho} = \frac{\mathbf{e}^{\theta} \times \mathbf{e}^{\zeta}}{\sqrt{g}}$, one [writes](https://github.com/PlasmaControl/DESC/blob/6d03cb015701b27d651bf804d36032c35119c536/desc/compute/_core.py#L426):
```python
data["e^rho"] = (cross(data["e_theta"], data["e_zeta"]).T / data["sqrt(g)"]).T
```
Note here that once the quantities are crossed, they are transposed. This is done to ensure that the result retains the desired shape of `(num_nodes,3)`.

### Be Mindful of Shapes

It is important to keep in mind the shapes of the quantities being manipulated to ensure the desired operation is carried out. As another example, the gradient of the magnetic toroidal flux $\nabla \psi = \frac{d\psi}{d\rho}\nabla \rho$ is [calculated as](https://github.com/PlasmaControl/DESC/blob/94d7e43542613b1c901fcd655502312f3e567c26/desc/compute/_core.py#L701):
```python
data["grad(psi)"] = (data["psi_r"] * data["e^rho"].T).T
```
The desired operation here is to multiply `data["psi_r"]`, which is a scalar quantity at each grid point and so is of shape `(num_nodes,)` with `data["e^rho"]`, a vector quantity and so is of shape `(num_nodes,3)`. 
We want the result to be of shape `(num_nodes,3)`. 
In order to do so, we first must transpose the vector quantity to be shape `(3,num_nodes)`, so that when multiplied together with the scalar quantity of shape `(num_nodes,3)`, the result is broadcast to an array of shape `(3,num_nodes)`. 
If the transpose did not occur, the two shapes `(num_nodes,)` and `(num_nodes,3)` would be incompatible with eachother.
The second transpose after the multiplication is to ensure that the result is in the shape `(num_nodes,3)`, as is the convention expected in the code.

## What `check_derivs()` does

Basically, this function ensures that the transforms passed to the compute function have the necessary derivatives of $R,Z,L,p,\iota$ to calculate the quantity contained in the if statement. 
If yes, it returns `True` and the quantity in the logival is computed. If not, it returns `False` and that quantitiy is not calculated. 
This allows us to call a function to get a specific quantity which may not need high order derivatives, and avoid needing to compute those derivatives anyways just to have the function call not throw an error that the necessary derivatives do not exist for a quantitiy we are not asking for but which needs higher order derivatives to compute

## `__init__.py`

`arg_order` is defined in this file. This tuple is used in other parts of the code to determine how to parse the state vector `x` into the various arguments that make it up, and also for making the derivatives of functions of these arguments, such as inside of `_set_derivatives` method of `_Objective` in `objective_funs.py`.

## `compute/utils.py`

 - dot
 - cross
 custom vector algebra fxns

# `configuration.py`

# `derivatives.py`

# `equilibrium.py`

# `examples`

# `geometry`

# `grid.py`

- explain symmetry and how that affects grid (delete lower half of poloidal plane)
 - show how it preserves FSA
- why L M need to be specific things for concentric grid to work

# `interpolate.py`

# `io`

# `magnetic_fields.py`

# `objectives`

talk about obj vs constraints
- [ ] why do we need to re-make the objectives when we change eq resolutoin
  - I think this is because the objectives get built for that eq (built = ?) , so remaking them means we get rid of the built status?
- is the size of the `x_reduced` equal to number of parameters? YES IT IS And is this equal to one of the sides of the linear constraint matrix `A`? it is in `factorize_linear_constraints` which is in `objectives.utils`, from `project` function

## `linear_objectives.py`

- when specifying interior surface as the fixboundary constraint, the self A becomes zernike_radial instead of 1?

## `objectives/utils.py`

#### `factorize_linear_constraints`

 - define problem in standard optimization setup
 - match which DESC variables are equal to the standard ones
 - write Ax=b
 - this thing basically finds the nullspace( A )=: Z
  - Z then can be multiplied with (x - x_particular) to obtain a vector which is guaranteed to be in the nullspace of the constraints, bc it is made up of a linear combinations of vectors which lie inside of the nullspace (does this make sense?)...

DESC approaches the ideal MHD fixed-boundary equilibrium problem $\mathbf{F}=0$ [as an optimization problem](https://desc-docs.readthedocs.io/en/latest/theory_general.html):

$$\begin{align}
\min_{\mathbf{x}\in \mathcal{R}^n} \mathbf{f}(\mathbf{x})&\\
\text{subject to the linear constraints}~~ \mathbf{A}\mathbf{x}=\mathbf{b}&
\end{align}$$

where the objective to be minimized is the MHD force balance error $\mathbf{F}=\mathbf{J}\times \mathbf{B} - \nabla p$, which is minimized by evaluating  the two components of $\mathbf{F}$ on a collocation grid (resulting in a vector of residuals $\mathbf{f} = [f_{\rho},f_{\beta}]$ of length `2*num_nodes` since each of $f_{\rho},f_{\beta}$ are evaluated at the collocation nodes) and then minimizing those residuals $\mathbf{f}(\mathbf{x})$. 
The state variable being minimized over  $\mathbf{x} = [R_{lmn}, Z_{lmn}, \lambda_{lmn}]$ is the vector of the Fourier-Zernike spectral coefficients used to describe the mapping between the toroidal $(R,\phi,Z)$ coordinates and the computational flux coordinates $(\rho,\theta,\zeta)$.
The state is of length `3*eq.R_basis.num_modes` (if a non-stellarator symmetric equilibrium, where the number of basis modes for R and Z are the same), or length `eq.R_basis.num_modes + 2 * eq.Z_basis.num_modes` (if a stellarator-symmetric equilibrium, where $R$ has $cos(m\theta-n\zeta)$ symmetry and $Z$ and $\lambda$ have $sin(m\theta-n\zeta)$).


A fixed-boundary equilbrium problem requires the fixed-boundary $R_b(\theta,\zeta),Z_b{\theta,\zeta}$ to be given as the input, and this appears as a linear constraint on $\mathbf{x}$ during the optimization. 
In DESC, additionally a [gauge constraint](https://desc-docs.readthedocs.io/en/latest/_api/objectives/desc.objectives.FixLambdaGauge.html) on $\lambda$ is applied, since $\lambda$ is only defined up to an additive multiple of $2\pi$, which constitutes another linear constraint to the problem. 
The linear constraints are then written in the form $\mathbf{A}\mathbf{x}=\mathbf{b}$. 

In solving for the equilibrium DESC deals with these linear constraints by using the feasible direction formulation (see for example [page 3 of this reference](https://www.cs.umd.edu/users/oleary/a607/607constr1hand.pdf)).

The state variable $\mathbf{x}$ is written as $\mathbf{x} = \mathbf{x}_p + \mathbf{Z}\mathbf{y}$

# `optimize`

# `perturbations.py`

# `plotting.py`
maybe pretty self-explanatory

# `transform.py`

 - what the transform is
  - has a basis and a grid, and evaluates that basis on its grid right? so it contains the transform matrices (call them transform matrices)? how are these called? but these are the matrices which, when multiplied against a vector of the coefficients of the spectral series, yields the series evaluated at the grid points

this file contains the `Transform` class. 

## `build()`

this method builds the transform matrices for each derivative order of the basis the transform requires (which is specified when the `Transform` object is initialized with the `derivs` argument).
Defining the transform matrix as $A_{(d\rho,d\theta,d\zeta)}$ for given derivatives of the basis ${(d\rho,d\theta,d\zeta)}$ (where each are integers), the matrix is used to transform a spectral basis with a given set of coefficients $\mathbf{c}$ into its values on the grid (given by `Transform.grid`) by 

$$ A\mathbf{c} = \mathbf{x}$$

where $\mathbf{x}$ is the values of the spectral basis evaluated at the grid points . 
$\mathbf{c}$ is a vector of length `Transform.basis.num_modes` (the number of modes in the basis), $\mathbf{x}$ is a vector of length `Transform.grid.num_nodes` (the number of nodes in the grid), and $A$ is a matrix of shape `(num_nodes,num_modes)`.


i.e. if a Fourier Series $f(\zeta) = 2 + 4*cos(\zeta)$, and the grid is $\zeta = (0,\pi)$, then $\mathbf{x} =\begin{bmatrix}0\\ \pi\end{bmatrix}$, $\mathbf{c}=\begin{bmatrix} 2\\ 4 \end{bmatrix}$, and  $A = \begin{bmatrix} 1 & cos(0)\\ 1& cos(\pi) \end{bmatrix} = \begin{bmatrix} 1& 1\\ 1& -1 \end{bmatrix} $ s.t. 
$$ A\mathbf{c} = \begin{bmatrix} 1& 1\\ 1& -1 \end{bmatrix} \begin{bmatrix} 2\\ 4 \end{bmatrix} = \begin{bmatrix} 6 \\ -2  \end{bmatrix}  $$


## `build_pinv()`

This function builds the pseudoinverse of the transform, which can then be used to take values of a given function that are given on `Transform.grid` and fit a spectral basis to them.
A couple different methods are available:

### `direct1`

With this method, 

# `utils.py`

# `vmec.py`

# `vmec_utils.py`